# 앙상블 검색기( Ensemble Retriever )
- Sparse retriever : 키워드 기반 ( BM25, TF-IDF, SPLADE )
- Dense retriever : 의미적 유사성 기반 ( DPR,ANCE , ColBERT )

두 결과 집합에 각각 점수를 부여한 후, 상호 순위 융합(Reciprocal Rank Fusion, RRF) 등 알고리즘을 써서 최종 결과로 합칩니다

- RRF 작동 원리
각 검색 시스템에서 리턴된 문서 순위의 역수 값을 점수로 부여합니다. 예를 들어, 순위 1위 문서의 점수는 1, 2위 문서는 1/2, 3위 문서는 1/3 식입니다.

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_ollama import OllamaLLM
from langchain import PromptTemplate
from langchain_community.document_loaders import TextLoader
from langchain.schema import Document
from langgraph.prebuilt import create_react_agent
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_community.document_transformers import LongContextReorder
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH08-Ensembleretriever")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH08-Ensembleretriever


In [ ]:
def get_split_docs():
    # 1. 텍스트 청크 분할 (chunk_size=1000, chunk_overlap=50)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

    # 2. Text loader
    loader = TextLoader("./data/text.txt", encoding="utf-8")
    return loader.load_and_split(text_splitter)

In [5]:
def get_retriever(split_docs): 
    model_name = "intfloat/multilingual-e5-large-instruct"
    hf_embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs={"device": "cuda"},  # cuda, cpu
        encode_kwargs={"normalize_embeddings": True},
    )
    db = FAISS.from_documents(documents=split_docs, embedding=hf_embeddings)
    retriever = db.as_retriever(
        search_type="mmr", search_kwargs={"k": 1, "lambda_mult": 0.25, "fetch_k": 20}
    )
    return retriever

In [6]:
docs = get_split_docs()

# bm25 retriever와 faiss retriever를 초기화합니다.
bm25_retriever = BM25Retriever.from_documents(
    docs
)
bm25_retriever.k = 1  # BM25Retriever의 검색 결과 개수를 1로 설정합니다.


faiss_retriever = get_retriever(docs)
# 앙상블 retriever를 초기화합니다.
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.7, 0.3],
)

In [8]:
query = "삼성전자 하반기 실적"
_docs = ensemble_retriever.invoke(query)
bm25_result = bm25_retriever.invoke(query)
faiss_result = faiss_retriever.invoke(query)
print("[Ensemble Retriever]")
for doc in _docs:
    print(f"Content: {doc.page_content}")
    print()
print("[BM25 Retriever]")
for doc in bm25_result:
    print(f"Content: {doc.page_content}")
    print()

print("[FAISS Retriever]")
for doc in faiss_result:
    print(f"Content: {doc.page_content}")
    print()

[Ensemble Retriever]
Content: 증권업계는 올해 '9만 전자' 달성도 가능하다고 전망한다. 16일 금융투자업계에 따르면, 키움증권(9만원), KB증권(9만원), BNK투자증권(9만1000원), NH투자증권(9만4000원), 한국투자증권(9만5000원), 미래에셋증권(9만6000원) 등이 삼성전자 목표가를 9만원 이상으로 제시했다. 
특히 NH투자증권은 16일 삼성전자 목표 주가를 기존보다 11.9% 상향했다. 류영호 NH투자증권 연구원은 16일 관련 보고서에서 "최근 파운드리 부문에서 테슬라와 애플 등 의미 있는 고객사가 확보된 점, 1c(10나노급 6세대) 수율 개선과 함께 하반기 엔비디아 고대역폭 메모리(HBM) 공급이 가능할 것이란 기대감 등에 힘입어 주가가 긍정적 흐름을 보이고 있다"며 이같이 밝혔다.

Content: 특히 파운드리 고객사를 확보한 것이 단기 실적에는 영향을 미치지 않지만, 중장기 경쟁력 회복을 위한 기반이 될 것이라고 평가했다. 류 연구원은 "여러 기대감이 모여 밸류에이션이 정상화되는 국면"이라고 분석했다. 
이어 3분기 영업이익은 9조원으로 1년 전보다는 1.9% 감소하지만 직전 분기 대비 91.9% 늘어날 것으로 낙관했다.
미래에셋증권은 15일 보고서에서 삼성전자의 올해와 내년 영업이익 전망치를 기존대비 각각 2.9%, 5.3%씩 올렸다. 동시에 목표 주가를 기존 8만8000원에서 9만6000원으로 9% 높였다.
미래에셋증권은 주가 상향 이유로 내년 메모리 반도체 공급 부족을 꼽았다. 반도체 업체들은 제한된 D램 생산라인을 갖고 있는데, 업황 불확실성 탓에 설비를 크게 늘리지 못했다. 새로 증설한 시설도 대부분 차세대 반도체인 고대역폭 메모리(HBM) 생산에 집중돼 있어 범용 D램의 생산량이 줄면서 가격이 오르고 있다.

[BM25 Retriever]
Content: 증권업계는 올해 '9만 전자' 달성도 가능하다고 전망한다. 16일 금융투자업계에 따르면, 키움증권(9만원), KB증권(9만원), BNK투자증권(